In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18
from PIL import Image
import os
import numpy as np
import tifffile as tiff


In [2]:
# SI-STSAR-7 Dataset
class SARImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir #initializing dataset by reading the root directory
        self.transform = transform #the transformations to be applied to the images
        self.classes = sorted(os.listdir(root_dir))  #sorts classes
        self.samples = [] #lists for corresponding labels
        #loop for each class folder in the root directory
        for label, class_dir in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_dir)
            #loop for each image in the class folder
            for fname in os.listdir(class_path):
                if fname.endswith(('tif')):
                    self.samples.append((os.path.join(class_path, fname), label))

    def __len__(self):
        return len(self.samples) #returns the total number of images

    def __getitem__(self, idx):
        img_path, label = self.samples[idx] #image and label at the specified index
        img = tiff.imread(img_path) #load tif image
        img = Image.fromarray(img) #convert tif image to PIL image

        if self.transform:
            img = self.transform(img) #apply the specified transformations to the image
        return img, label


In [3]:
# dataset direction
data_dir = 'only1img4allHHc5 - Copy'

# Data augmentation
data_augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10)
])

# Preprocessing
preprocess_transform = transforms.Compose([
    transforms.ToTensor(),
])

# Applying data augmentation and preprocessing to only training dataset
train_transform = transforms.Compose([
    data_augmentation,
    preprocess_transform
])

# Load the dataset
full_dataset = SARImageDataset(root_dir=data_dir, transform=preprocess_transform)


# Perform a 70-15-15 train-val-test split on the original dataset length
dataset_size = len(full_dataset)
train_size = int(0.6 * dataset_size)
val_size = int(0.2 * dataset_size)
test_size = dataset_size - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size]) # split the datasets

#data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Print the sizes of the datasets
print(f"Total number of samples in the full dataset: {len(full_dataset)}")
print(f"Number of samples in the training set: {len(train_dataset)}")
print(f"Number of samples in the testing set: {len(test_dataset)}")

# print the classes
def print_class(root_dir):
    class_names = sorted(os.listdir(root_dir)) #sorts and lists all folders in the main folder
    for i, class_name in enumerate(class_names): #prints each class with folder names
        print(f"Class {i}: {class_name}")
print_class(data_dir)


Total number of samples in the full dataset: 164564
Number of samples in the training set: 98738
Number of samples in the testing set: 32914
Class 0: GI_6s - GWI_6s - NI_6s
Class 1: MediumFI_6s
Class 2: OW_6s
Class 3: ThickFI_6s
Class 4: ThinFI_6s


# ResNet

In [5]:
# Training, Validation, and Testing
from tqdm import tqdm #for time counting

def train(train_loader, val_loader, model, criterion, optimizer, num_epochs, device):
    model.to(device)
    best_val_accuracy = 0.0
    best_model = None
    
    for epoch in range(num_epochs):
        model.train() #set model for training mode
        running_loss = 0.0 #initializing running loss
        correct = 0 #for time counting
        total = 0 #for time counting
        #progress bar for time counting
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for i, (images, labels) in progress_bar:
            images, labels = images.to(device), labels.to(device)  #data move to the appropriate device/GPU
            optimizer.zero_grad() #all gradients to zero
            outputs = model(images)
            loss = criterion(outputs, labels) #calculate loss
            loss.backward() #computing the gradient of loss with respect to each parameters in model
            optimizer.step() #updating model
            
            running_loss += loss.item() * images.size(0) #accumulating loss
            
            #time counting
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            progress_bar.set_postfix(loss=running_loss/(i+1), accuracy=100.*correct/total)
        epoch_accuracy = 100. * correct / total #accuracy per epoch
        
        epoch_loss = running_loss / len(train_loader.dataset) #avg loss computation
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.2f}, Accuracy: {epoch_accuracy:.2f}%')

        #validate the model
        val_accuracy = validate(val_loader, model, device)
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()

    model.load_state_dict(best_model)  #load the best model
    print("Training complete")
    return model


def test(test_loader, model, device):
    model.to(device)
    model.eval() #set evaluation mode
    correct = 0 #correct predictions
    total = 0 #total samples
    
    with torch.no_grad():  #iterating over the test dataset without tracking gradients
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  #move data to the appropriate device
            
            outputs = model(images)  #forward pass

            _, predicted = torch.max(outputs, 1)  #get the index of the max log-probability (prediction)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = correct / total
    print(f'Test Accuracy %: {accuracy * 100:.2f}%')
    return accuracy


def validate(val_loader, model, device):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f'Validation Accuracy %: {accuracy * 100:.2f}%')
    return accuracy

#define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #for "RuntimeError: Found no NVIDIA driver on your system." error

In [4]:
#trying ResNet model

# Load the pre-trained ResNet18 model
model = resnet18(pretrained=True) #model weights pre-trained on the ImageNet dataset, "True" gave better results than "False", about 1-2%

# Redefine first layer (single-band input)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) #RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[32, 1, 32, 32] to have 3 channels, but got 1 channels instead
#the data does not have 3 channel, it's single band HH. so first value should be "1"

# Disable gradients from all layers of the model
for param in model.parameters():
    param.requires_grad = False

# replace the last layer with a new linear layer where gradients are active
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5) #5 classes

# enable gradients for the new layer
for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
num_epochs = 10

# turning on gradient calculation for all layers
for param in model.parameters():
    param.requires_grad = True

#the optimizer to optimize all parameters
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# train the model for fine-tuning
fine_tuned_model = train(train_loader, val_loader, model, criterion, optimizer, num_epochs, device)

# test the model for fine-tuning
fine_tuned_accuracy = test(test_loader, fine_tuned_model, device)

Epoch 1/10: 100%|████████████████████████████████████████| 3086/3086 [04:05<00:00, 12.55it/s, accuracy=57.6, loss=33.3]


Epoch 1/10, Loss: 1.04, Accuracy: 57.63%
Validation Accuracy %: 60.82%


Epoch 2/10: 100%|████████████████████████████████████████| 3086/3086 [04:15<00:00, 12.07it/s, accuracy=59.4, loss=31.8]


Epoch 2/10, Loss: 0.99, Accuracy: 59.44%
Validation Accuracy %: 60.99%


Epoch 3/10: 100%|████████████████████████████████████████| 3086/3086 [04:14<00:00, 12.12it/s, accuracy=59.9, loss=31.2]


...

Epoch 8/10, Loss: 0.92, Accuracy: 62.14%
Validation Accuracy %: 60.94%


Epoch 9/10: 100%|████████████████████████████████████████| 3086/3086 [03:52<00:00, 13.27it/s, accuracy=62.6, loss=29.2]


Epoch 9/10, Loss: 0.91, Accuracy: 62.55%
Validation Accuracy %: 56.46%


Epoch 10/10: 100%|███████████████████████████████████████| 3086/3086 [04:12<00:00, 12.22it/s, accuracy=62.9, loss=28.8]


Epoch 10/10, Loss: 0.90, Accuracy: 62.90%
Validation Accuracy %: 52.90%
Training complete
Test Accuracy %: 52.79%


When low accuracy was seen with ResNet, which gave the fastest results, no time was wasted by continuing with EfficientNet. It is obvious that it will give similar low results.